In [41]:
# import packages
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, utils
from torch.utils.data import TensorDataset
from tqdm import tqdm
from torch_geometric.nn import TransformerConv, global_max_pool, GATv2Conv, ClusterGCNConv, PointTransformerConv, global_mean_pool, dense_mincut_pool
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
import time
import torch 
from torch_geometric.data import Data, InMemoryDataset, download_url
import torch_geometric.transforms as T
import networkx as nx
from torch_cluster import knn_graph
from torch_geometric.utils import from_networkx, to_dense_adj, dense_to_sparse, to_dense_batch

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [43]:
# Dataset class
import torch
from torch_geometric.data import Data, InMemoryDataset, download_url
import torch_geometric.transforms as T

class Quark_Gluon_Dataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(Quark_Gluon_Dataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['data.pt']

    def download(self):
        pass

    def create_graph(self, image):
        G = nx.Graph()
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                if image[i][j].any() > 0:
                    #add node (i,j) to graph with attribute 'x' = image[i][j] and pos = (i,j)
                    G.add_node((i,j), x=image[i][j], pos=(i,j,0))
                    if i > 0 and image[i-1][j].any() > 0:
                        G.add_edge((i,j), (i-1,j))
                    if j > 0 and image[i][j-1].any() > 0:
                        G.add_edge((i,j), (i,j-1))
                    if i < image.shape[0]-1 and image[i+1][j].any() > 0:
                        G.add_edge((i,j), (i+1,j))
                    if j < image.shape[1]-1 and image[i][j+1].any() > 0:
                        G.add_edge((i,j), (i,j+1))
                    if i > 0 and j > 0 and image[i-1][j-1].any() > 0:
                        G.add_edge((i,j), (i-1,j-1))
                    if i < image.shape[0]-1 and j < image.shape[1]-1 and image[i+1][j+1].any() > 0:
                        G.add_edge((i,j), (i+1,j+1))
                    if i > 0 and j < image.shape[1]-1 and image[i-1][j+1].any() > 0:
                        G.add_edge((i,j), (i-1,j+1))
                    if i < image.shape[0]-1 and j > 0 and image[i+1][j-1].any() > 0:
                        G.add_edge((i,j), (i+1,j-1))

        return G

    def process(self):
        f = h5py.File('/hdfs1/Data/Shrutimoy/quark-gluon_data-set_n139306.hdf5', 'r')
        X_jets = np.asarray(f['X_jets'])
        m0 = np.asarray(f['m0'])
        pt = np.asarray(f['pt'])
        y = np.asarray(f['y'])
        data_list = []
        for i in tqdm(range(len(X_jets))):
            G = self.create_graph(X_jets[i])
            data = from_networkx(G)
            data.y = torch.tensor(y[i], dtype=torch.long)
            data.m = torch.tensor(m0[i], dtype=torch.float)
            data.p = torch.tensor(pt[i], dtype=torch.float)
            data_list.append(data)

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [44]:
def train_test_split(X, y, test_size):
    dataset_size = len(X)
    train_data_X = X[:int(dataset_size*(1-test_size))]
    test_data_X = X[int(dataset_size*(1-test_size)):]
    train_data_y = y[:int(dataset_size*(1-test_size))]
    test_data_y = y[int(dataset_size*(1-test_size)):]
    return train_data_X, test_data_X, train_data_y, test_data_y


In [109]:
#loss function for Graph Autoencoder
def loss_function(recon_x, x, mu, logvar, mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss):
    #print(recon_x.shape, x.shape)
    BCE = F.mse_loss(recon_x, x)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD + mc1_loss + o1_loss + mc2_loss + o2_loss + mc3_loss + o3_loss

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.Sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.Sum += val * n
        self.count += n
        self.avg = self.Sum / self.count

In [110]:
#Graph Autoencoder Model
class GAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, cluster_dim, dropout, batch_size):
        super(GAE, self).__init__()
        self.batch_size = batch_size
        self.fc0 = nn.Linear(input_dim+3, hidden_dim[0])
        self.batch_norm0 = nn.BatchNorm1d(hidden_dim[0])
        self.conv1 = nn.ModuleList()
        self.fc11 = nn.Linear(hidden_dim[0]*2, hidden_dim[1])
        self.batch_norm11 = nn.BatchNorm1d(hidden_dim[1])
        self.mlp1 = nn.Sequential()
        self.conv2 = nn.ModuleList()
        self.fc12 = nn.Linear(hidden_dim[1]*2, hidden_dim[2])
        self.batch_norm12 = nn.BatchNorm1d(hidden_dim[2])
        self.mlp2 = nn.Sequential()
        self.conv3 = nn.ModuleList()
        self.fc13 = nn.Linear(hidden_dim[2]*2, hidden_dim[3])
        self.batch_norm13 = nn.BatchNorm1d(hidden_dim[3])
        self.mlp3 = nn.Sequential()
        self.conv4 = nn.ModuleList()
        self.fc14 = nn.Linear(hidden_dim[3]*2, hidden_dim[4])
        self.batch_norm14 = nn.BatchNorm1d(hidden_dim[4])

        self.conv1.append(GATv2Conv(hidden_dim[0], hidden_dim[0], add_self_loops=True, dropout=dropout))
        self.conv1.append(ClusterGCNConv(hidden_dim[0], hidden_dim[0], add_self_loops=True, dropout=dropout))
        self.mlp1.append(nn.Linear(hidden_dim[1], hidden_dim[1]))
        self.mlp1.append(nn.ReLU())
        self.mlp1.append(nn.Linear(hidden_dim[1], cluster_dim[0]))

        self.conv2.append(GATv2Conv(hidden_dim[1], hidden_dim[1], add_self_loops=True, dropout=dropout, edge_dim=1))
        self.conv2.append(ClusterGCNConv(hidden_dim[1], hidden_dim[1], add_self_loops=True, dropout=dropout))
        self.mlp2.append(nn.Linear(hidden_dim[2], hidden_dim[2]))
        self.mlp2.append(nn.ReLU())
        self.mlp2.append(nn.Linear(hidden_dim[2], cluster_dim[1]))
    
        self.conv3.append(GATv2Conv(hidden_dim[2], hidden_dim[2], add_self_loops=True, dropout=dropout, edge_dim=1))
        self.conv3.append(ClusterGCNConv(hidden_dim[2], hidden_dim[2], add_self_loops=True, dropout=dropout))
        self.mlp3.append(nn.Linear(hidden_dim[3], hidden_dim[3]))
        self.mlp3.append(nn.ReLU())
        self.mlp3.append(nn.Linear(hidden_dim[3], cluster_dim[2]))

        self.conv4.append(GATv2Conv(hidden_dim[3], hidden_dim[3], add_self_loops=True, dropout=dropout, edge_dim=1))
        self.conv4.append(ClusterGCNConv(hidden_dim[3], hidden_dim[3], add_self_loops=True, dropout=dropout))

        self.fc1 = nn.Linear(hidden_dim[4], hidden_dim[5])
        self.fc2 = nn.Linear(hidden_dim[4], hidden_dim[5])
        self.fc3 = nn.Linear(hidden_dim[5], hidden_dim[4])

        self.conv5 = nn.ModuleList()
        self.fc21 = nn.Linear(hidden_dim[4]*2, hidden_dim[3])
        self.batch_norm21 = nn.BatchNorm1d(hidden_dim[3])
        self.conv6 = nn.ModuleList()
        self.fc22 = nn.Linear(hidden_dim[3]*2, hidden_dim[2])
        self.batch_norm22 = nn.BatchNorm1d(hidden_dim[2])
        self.conv7 = nn.ModuleList()
        self.fc23 = nn.Linear(hidden_dim[2]*2, hidden_dim[1])
        self.batch_norm23 = nn.BatchNorm1d(hidden_dim[1])
        self.conv8 = nn.ModuleList()
        self.fc24 = nn.Linear(hidden_dim[1]*2, hidden_dim[0])
        self.batch_norm24 = nn.BatchNorm1d(hidden_dim[0])

        self.conv5.append(GATv2Conv(hidden_dim[4], hidden_dim[4], add_self_loops=True, dropout=dropout, edge_dim=1))    
        self.conv5.append(ClusterGCNConv(hidden_dim[4], hidden_dim[4], add_self_loops=True, dropout=dropout))
        

        self.conv6.append(GATv2Conv(hidden_dim[3], hidden_dim[3], add_self_loops=True, dropout=dropout, edge_dim=1))
        self.conv6.append(ClusterGCNConv(hidden_dim[3], hidden_dim[3], add_self_loops=True, dropout=dropout))
        

        self.conv7.append(GATv2Conv(hidden_dim[2], hidden_dim[2], add_self_loops=True, dropout=dropout, edge_dim=1))
        self.conv7.append(ClusterGCNConv(hidden_dim[2], hidden_dim[2], add_self_loops=True, dropout=dropout))
        

        self.conv8.append(GATv2Conv(hidden_dim[1], hidden_dim[1], add_self_loops=True, dropout=dropout, edge_dim=1))
        self.conv8.append(ClusterGCNConv(hidden_dim[1], hidden_dim[1], add_self_loops=True, dropout=dropout))
        

        self.fc4 = nn.Linear(hidden_dim[0], input_dim)

    def encode(self, x, edge_index, pos, batch):
        #print("encoder start")
        pos = pos.float()
        x = torch.cat((x, pos), dim=1)
        
        #print(x.shape)
        x = self.fc0(x)
        x = F.relu(x)
        x = self.batch_norm0(x)
        #print("1st layer")
        x1 = self.conv1[0](x, edge_index)
        x3 = self.conv1[1](x, edge_index)
        x = torch.cat((x1, x3), dim=1)
        x = self.fc11(x)
        x = F.relu(x)
        x = self.batch_norm11(x)
        adj = to_dense_adj(edge_index, max_num_nodes=x.size(0))
        x_new, mask = to_dense_batch(x)
        s1 = self.mlp1(x_new)
        x, adj, mc1_loss, o1_loss = dense_mincut_pool(x_new, adj, s1, mask)
        x = x.reshape(x.size(0)*x.size(1), -1)
        edge_index, edge_attr = dense_to_sparse(adj)

        #print("2nd layer")  
        x1 = self.conv2[0](x, edge_index, edge_attr=edge_attr)
        x3 = self.conv2[1](x, edge_index)
        x = torch.cat((x1, x3), dim=1)
        x = self.fc12(x)
        x = F.relu(x)
        x = self.batch_norm12(x)
        adj = to_dense_adj(edge_index, max_num_nodes=x.size(0))
        x_new, mask = to_dense_batch(x)
        s2 = self.mlp2(x_new)
        x, adj, mc2_loss, o2_loss = dense_mincut_pool(x_new, adj, s2, mask)
        edge_index, edge_attr = dense_to_sparse(adj)

        #print("3rd layer")
        x = x.reshape(x.size(0)*x.size(1), -1)
        x1 = self.conv3[0](x, edge_index, edge_attr=edge_attr)
        x3 = self.conv3[1](x, edge_index)
        x = torch.cat((x1, x3), dim=1)
        x = self.fc13(x)  
        x = F.relu(x)
        x = self.batch_norm13(x)
        adj = to_dense_adj(edge_index, max_num_nodes=x.size(0))
        x_new, mask = to_dense_batch(x)
        s3 = self.mlp3(x_new)
        x, adj, mc3_loss, o3_loss = dense_mincut_pool(x_new, adj, s3, mask)
        edge_index, edge_attr = dense_to_sparse(adj)
        last_edge = edge_index
        last_attr = edge_attr

        #print("4th layer")
        x = x.reshape(x.size(0)*x.size(1), -1)
        x1 = self.conv4[0](x, edge_index, edge_attr=edge_attr)
        x3 = self.conv4[1](x, edge_index)
        x = torch.cat((x1, x3), dim=1)
        x = self.fc14(x)
        x = F.relu(x)
        x = self.batch_norm14(x)
        #print("encoder end")
        return self.fc1(x), self.fc2(x), mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss, s1, s2, s3, last_edge, last_attr
    
    def reparameterize(self, mu, sigma):
        #print("in reparameterize")
        std = sigma.mul(0.5).exp_()
        eps = torch.FloatTensor(std.size()).normal_().to(device)
        eps = Variable(eps)
        #print("reparameterize end")
        return eps.mul(std).add_(mu)
    
    def decode(self, z, last_edge, last_attr, S1, S2, S3):
        #print("in decode")
        z = self.fc3(z)
        z1 = self.conv5[0](z, last_edge, last_attr)
        z2 = self.conv5[1](z, last_edge)
        z = torch.cat((z1, z2), dim=1)
        z = self.fc21(z)
        z = F.relu(z)
        z = self.batch_norm21(z)
        s = torch.squeeze(S3)
        z = torch.matmul(s, z)
        adj = to_dense_adj(last_edge, edge_attr=last_attr, max_num_nodes=z.size(0))
        #print(adj.shape, S[2].reshape(S[2].shape[1], S[2].shape[2]).shape)
        adj_pool = torch.matmul(s.T, adj)
        #print(adj_pool.shape)
        adj_pool = torch.matmul(adj_pool, s)
        edge_index, edge_attr = dense_to_sparse(adj_pool)
 
        z1 = self.conv6[0](z, edge_index, edge_attr)
        z2 = self.conv6[1](z, edge_index)
        z = torch.cat((z1, z2), dim=1)
        z = self.fc22(z)
        z = F.relu(z)
        z = self.batch_norm22(z)
        s = torch.squeeze(S2)
        z = torch.matmul(s, z)
        adj = to_dense_adj(edge_index, max_num_nodes=z.size(0))
        adj_pool = torch.matmul(s.T, adj)
        adj_pool = torch.matmul(adj_pool, s)
        edge_index, edge_attr = dense_to_sparse(adj_pool)

        z1 = self.conv7[0](z,edge_index, edge_attr)
        z2 = self.conv7[1](z, edge_index)
        z = torch.cat((z1, z2), dim=1)
        z = self.fc23(z)
        z = F.relu(z)
        z = self.batch_norm23(z)
        s = torch.squeeze(S1)
        z = torch.matmul(s, z)
        adj = to_dense_adj(edge_index, max_num_nodes=z.size(0))
        adj_pool = torch.matmul(s.T, adj)
        adj_pool = torch.matmul(adj_pool, s)
        edge_index, edge_attr = dense_to_sparse(adj_pool)

        z1 = self.conv8[0](z, edge_index, edge_attr)
        z2 = self.conv8[1](z, edge_index)
        z = torch.cat((z1, z2), dim=1)
        z = self.fc24(z)
        z = F.relu(z)
        z = self.batch_norm24(z)
        
        z = self.fc4(z)
        #print(z.shape)
        #print("out decode")
        z = F.softmax(z)
        return z
    
    def forward(self, data):
        x, edge_index, pos, batch = data.x, data.edge_index, data.pos, data.batch
        mu, sigma, mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss, s1, s2, s3, last_edge, last_attr = self.encode(x, edge_index, pos, batch)
        z = self.reparameterize(mu, sigma)
        #print(S[0].shape, S[1].shape, S[2].shape)
        x_new = self.decode(z, last_edge, last_attr, s1, s2, s3)
        return x_new, mu, sigma, mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss

In [47]:
dataset = Quark_Gluon_Dataset(root='/hdfs1/Data/Shubhajit/Quark_Gluon_Data_1/')

In [48]:
# split dataset into train, validation and test
train_dataset = dataset[:int(0.8*len(dataset))]
val_dataset = dataset[int(0.8*len(dataset)):int(0.9*len(dataset))]
test_dataset = dataset[int(0.9*len(dataset)):]

# load the dataset
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [113]:
#initialize the model
model = GAE(3, [16, 32, 64, 128, 256, 512], [256, 128, 64], 0, 16).to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.50)

In [114]:
loss_train, loss_test, loss_val = [], [], []
#traing loop
def train():
    model.train()
    train_loss = 0
    step = 0
    for data in tqdm(train_loader):
        optimizer.zero_grad()
        data = data.to(device)
        #copy data.x tensor to another tensor
        orig_x = data.x.clone().detach()
        #print(data.x.shape, orig_x.shape)
        z, mu, sigma, mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss = model(data)
        #print(data.x.shape, orig_x.shape, z.shape)
        loss = loss_function(z, orig_x, mu, sigma, mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss)
        loss.backward()
        train_loss += loss.item()
        #print(loss.item())
        optimizer.step()
        step += 1
        if step%100 == 0:
            print("Epoch: {} Train loss: {:.4f}".format(
          epoch+1, train_loss/(step*16)))
    return train_loss/len(train_loader.dataset)
    
    
#validation loop
def validation():
    model.eval()
    val_loss = 0
    for data in tqdm(val_loader):
        data = data.to(device)
        orig_x = data.x.clone().detach()
        z, mu, sigma, mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss = model(data)
        loss = loss_function(z, orig_x, mu, sigma, mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss)
        val_loss += loss.item()
    return val_loss/len(val_loader.dataset)

    
#validation loop
def test():
    model.eval()
    test_loss = 0
    for data in tqdm(test_loader):
        data = data.to(device)
        orig_x = data.x.clone().detach()
        z, mu, sigma, mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss = model(data)
        loss = loss_function(z, orig_x, mu, sigma, mc1_loss, o1_loss, mc2_loss, o2_loss, mc3_loss, o3_loss)
        test_loss += loss.item()
    return test_loss/len(test_loader.dataset)

#training loop
for epoch in range(10):
    train_loss = train()
    val_loss = validation()
    test_loss = test()
    #scheduler.step()
    print('====> Epoch: {} Train loss: {:.4f}'.format(
          epoch+1, train_loss))
    print('====> Epoch: {} Validation loss: {:.4f}'.format(
          epoch+1, val_loss))
    print('====> Epoch: {} Test loss: {:.4f}'.format(
            epoch+1, test_loss))
    loss_train.append(train_loss)
    loss_test.append(test_loss)
    loss_val.append(val_loss)

  1%|▏         | 100/6966 [00:32<39:32,  2.89it/s]

Epoch: 1 Train loss: 19.2400


  3%|▎         | 200/6966 [01:06<40:18,  2.80it/s]

Epoch: 1 Train loss: 9.6792


  4%|▍         | 300/6966 [01:39<38:15,  2.90it/s]

...

  6%|▌         | 400/6966 [02:14<37:15,  2.94it/s]

Epoch: 2 Train loss: 0.0881


  7%|▋         | 500/6966 [02:47<33:55,  3.18it/s]

Epoch: 2 Train loss: 0.0883


  9%|▊         | 598/6966 [03:20<35:53,  2.96it/s]